In [ ]:
!pip install langchain chromadb sentence-transformers huggingface_hub transformers PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 1.9 MB/s eta 0:0

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [ ]:
!pip install -U langchain-huggingface

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR_HF_API_TOKEN"

In [ ]:
from huggingface_hub import login
login("YOUR_HF_API_TOKEN")

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Cerebral palsy detection.pdf to Cerebral palsy detection.pdf


In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader(next(iter(uploaded)))
pages = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
docs = splitter.split_documents(pages)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_db")

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint
import os

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
    temperature=0.3,
    max_new_tokens=512,
    task = "text-generation"
)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

custom_prompt = PromptTemplate.from_template("""
You are an AI assistant. Use the below document excerpts to answer the user's question.
If the answer is not in the documents, say "I could not find that in the provided context."

Context:
{context}

Question: {question}
Answer:
""")

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": custom_prompt}
)

In [ ]:
docs = retriever.get_relevant_documents("Which dataset is used in the paper?")
for i, doc in enumerate(docs):
    print(f"\n--- Retrieved Chunk {i+1} ---\n{doc.page_content}")


--- Retrieved Chunk 1 ---
Our proposed methodology also introduces a shift in the treatment of data structure, 
aiming to optimize the training process and elevate the overall performance of the 
model. In the preceding architecture, where each extracted feature (e.g., HOJO2D, 
HOJD2D, FFT-JD, and FFT-JO) encapsulates information pertaining to all eight 
distinct joints, a crucial reformatting initiative is taken. This reformatting strategy

--- Retrieved Chunk 2 ---
S. Rajpopat et al.
  106  Page 8 of 19
dataset availability, addressing imbalance issues, and conducting thorough evalua-
tions on challenging datasets to ensure robustness and real-world applicability, thus 
advancing this field and improving the safety of children affected by cerebral palsy. 
On the basis of the drawbacks we have found in the previous works, we have derived 
the following objectives:
• To analyze body part-based features in the time and frequency domain for cere-

--- Retrieved Chunk 3 ---
datasets to c

In [ ]:
query = "Is there a dataset mentioned in this document?" "List the datasets mentioned in the paper."
response = qa_chain.invoke(query)
print("Answer:", response)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Answer: {'query': 'Is there a dataset mentioned in this document?List the datasets mentioned in the paper.', 'result': '• MINI-RGBD\n• RVI-38'}
